### Instalar librerías y reiniciar kernel.

In [1]:
pip install mysql-connector

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


### Importaciones

In [3]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import *
import json
import numpy as np

### Crear acceso a engine de mysql e inspeccionar.

In [4]:
engine = create_engine('mysql+mysqlconnector://test:test123@db/test', echo = True)

In [5]:
pd.read_sql("select current_date", con=engine)

2021-02-04 06:48:17,340 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-02-04 06:48:17,342 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:48:17,348 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-02-04 06:48:17,351 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:48:17,356 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2021-02-04 06:48:17,358 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:48:17,365 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2021-02-04 06:48:17,367 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:48:17,372 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2021-02-04 06:48:17,374 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:48:17,381 INFO sqlalchemy.engine.base.Engine DESCRIBE `select current_date`
2021-02-04 06:48:17,384 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:48:17,392 INFO sqlalc

,current_date
0,2021-02-04


In [6]:
inspector = inspect(engine)
schemas = inspector.get_schema_names()

for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        for column in inspector.get_columns(table_name, schema=schema):
            print("Column: %s" % column)

2021-02-04 06:48:17,472 INFO sqlalchemy.engine.base.Engine SHOW schemas
2021-02-04 06:48:17,474 INFO sqlalchemy.engine.base.Engine {}
schema: information_schema
2021-02-04 06:48:17,481 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `information_schema`
2021-02-04 06:48:17,483 INFO sqlalchemy.engine.base.Engine {}
schema: test
2021-02-04 06:48:17,493 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `test`
2021-02-04 06:48:17,495 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:48:17,543 INFO sqlalchemy.engine.base.OptionEngine SHOW CREATE TABLE `test`.`unconfirmed_transactions`
2021-02-04 06:48:17,545 INFO sqlalchemy.engine.base.OptionEngine {}
Column: {'name': 'index', 'type': BIGINT(display_width=20), 'default': None, 'comment': None, 'nullable': True, 'autoincrement': False}
Column: {'name': 'lock_time', 'type': BIGINT(display_width=20), 'default': None, 'comment': None, 'nullable': True, 'autoincrement': False}
Column: {'name': 'ver', 'type': BIGINT(display_widt

In [7]:
print(engine.table_names())

2021-02-04 06:48:17,586 INFO sqlalchemy.engine.base.Engine SHOW FULL TABLES FROM `test`
2021-02-04 06:48:17,588 INFO sqlalchemy.engine.base.Engine {}
['unconfirmed_transactions', 'vaccinations_per_country']


# ETL 3

## Extraer

In [8]:
df = pd.read_csv('sources/time_series_covid19_confirmed_global.csv')
df.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,46837,47072,47306,47516,47716,47851,48053,48116,48229,48527
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,41302,42148,42988,43683,44436,45188,46061,46863,47742,48530
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,86730,87502,88252,88825,89416,90014,90579,91121,91638,92102


In [9]:
df.dtypes

Province/State     object
Country/Region     object
Lat               float64
Long              float64
1/22/20             int64
                   ...   
12/9/20             int64
12/10/20            int64
12/11/20            int64
12/12/20            int64
12/13/20            int64
Length: 331, dtype: object

## Transformar

In [10]:
df = df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
             var_name=["date"],
             value_name="cases")
df.head()

,Province/State,Country/Region,Lat,Long,date,cases
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [11]:
df['date'] =  pd.to_datetime(df['date'], format='%m/%d/%y')
df.head()

,Province/State,Country/Region,Lat,Long,date,cases
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0
1,NaN,Albania,41.15330,20.168300,2020-01-22,0
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0


In [12]:
pd.isna(df["Province/State"])

0        True
1        True
2        True
3        True
4        True
         ... 
88612    True
88613    True
88614    True
88615    True
88616    True
Name: Province/State, Length: 88617, dtype: bool

In [13]:
df = df.astype(object).replace(np.nan, 'None')

In [14]:
u = df.select_dtypes(object)
df[u.columns] = u.apply(
    lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

## Load al engine

In [15]:
create_table_query = pd.io.sql.get_schema(df.reset_index(), 'test.time_series_covid19_confirmed_global', con=engine)
print(create_table_query)
type(create_table_query)


CREATE TABLE `test.time_series_covid19_confirmed_global` (
	`index` BIGINT, 
	`Province/State` TEXT, 
	`Country/Region` TEXT, 
	`Lat` TEXT, 
	`Long` TEXT, 
	date DATETIME, 
	cases BIGINT
)




str

In [16]:
with open("schemas/time_series_covid19_confirmed_global.sql", "w") as text_file:
    text_file.write(create_table_query)

In [17]:
if not engine.dialect.has_table(engine, "test.time_series_covid19_confirmed_global"):  # If table don't exist, Create.
    print("test.time_series_covid19_confirmed_global no existe")
    df.to_sql('time_series_covid19_confirmed_global', con=engine, schema="test", if_exists='append')

2021-02-04 06:48:22,729 INFO sqlalchemy.engine.base.OptionEngine DESCRIBE `test.time_series_covid19_confirmed_global`
2021-02-04 06:48:22,731 INFO sqlalchemy.engine.base.OptionEngine {}
2021-02-04 06:48:22,734 INFO sqlalchemy.engine.base.OptionEngine ROLLBACK
test.time_series_covid19_confirmed_global no existe
2021-02-04 06:48:22,865 INFO sqlalchemy.engine.base.Engine DESCRIBE `test`.`time_series_covid19_confirmed_global`
2021-02-04 06:48:22,866 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:48:22,871 INFO sqlalchemy.engine.base.Engine ROLLBACK
2021-02-04 06:48:22,881 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE test.time_series_covid19_confirmed_global (
	`index` BIGINT, 
	`Province/State` TEXT, 
	`Country/Region` TEXT, 
	`Lat` TEXT, 
	`Long` TEXT, 
	date DATETIME, 
	cases BIGINT
)


2021-02-04 06:48:22,883 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:48:23,074 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-04 06:48:23,077 INFO sqlalchemy.engine.base.Engine CREATE I

In [18]:
with engine.begin() as transaction:
    transaction.execute('DELETE FROM test.time_series_covid19_confirmed_global')
    df.to_sql('time_series_covid19_confirmed_global', con=transaction, schema='test', if_exists='append', index=False)

2021-02-04 06:48:44,562 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-04 06:48:44,565 INFO sqlalchemy.engine.base.Engine DELETE FROM test.time_series_covid19_confirmed_global
2021-02-04 06:48:44,567 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:48:44,905 INFO sqlalchemy.engine.base.Engine DESCRIBE `test`.`time_series_covid19_confirmed_global`
2021-02-04 06:48:44,907 INFO sqlalchemy.engine.base.Engine {}
2021-02-04 06:48:47,998 INFO sqlalchemy.engine.base.Engine INSERT INTO test.time_series_covid19_confirmed_global (`Province/State`, `Country/Region`, `Lat`, `Long`, date, cases) VALUES (%(Province/State)s, %(Country/Region)s, %(Lat)s, %(Long)s, %(date)s, %(cases)s)
2021-02-04 06:48:48,004 INFO sqlalchemy.engine.base.Engine ({'Province/State': 'None', 'Country/Region': 'Afghanistan', 'Lat': None, 'Long': None, 'date': datetime.datetime(2020, 1, 22, 0, 0), 'cases': 0}, {'Province/State': 'None', 'Country/Region': 'Albania', 'Lat': None, 'Long': None, 'date': datetime.d